# Decorator #1 - Memoization

Classic example - Fibonacci series
---

In [1]:
def fibonacci(n):
    if n<2:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)
    
%timeit fibonacci(30) 

448 ms ± 9.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fibonacci_c(n):
    if n<2:
        return 1
    return fibonacci_c(n-1) + fibonacci_c(n-2)

%timeit fibonacci_c(30) 

126 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Levinstein distance
---

The minimal number of deletion & insertions between two words

### Example:

"reshape"$\rightarrow$"~~r~~eshape"$\rightarrow$"~~re~~shape"$\rightarrow$"~~re~~shape**s**"$\rightarrow$"shapes"

__Distance__:
    1. Two Deletions (Charaters r & e)
    2. One Insertion (s)
    3. Levinstein Distance of 3


In [3]:
def levinstein(a, b):
    if a==b:
        return 0
    if not any(a):
        return len(b)
    if not any(b):
        return len(a)
    if a[0]==b[0]:
        return levinstein(a[1:],b[1:])
    return 1+min(levinstein(a,b[1:]), levinstein(a[1:],b))
    

Test on real data
---

In [37]:
with open("/usr/share/dict/words", 'r') as f:
    words = list(filter(lambda w: len(w)>2,map(lambda w: w.strip().lower(), f.readlines())))
import random
random_words = random.sample(words, 10)
random_words

['desegmented',
 'aloneness',
 'pundit',
 'klip',
 'hanbalite',
 'asellate',
 'phytoserological',
 'affiant',
 'antireforming',
 'ciconiae']

In [5]:
@lru_cache(maxsize=None)
def levinstein_cached(a, b):
    if a==b:
        return 0
    if not any(a):
        return len(b)
    if not any(b):
        return len(a)
    if a[0]==b[0]:
        return levinstein_cached(a[1:],b[1:])
    return 1+min(levinstein_cached(a,b[1:]), levinstein_cached(a[1:],b))

In [6]:
def time_on_random_words(f):
    import time
    startTime = int(round(time.time() * 1000))
    for w1 in random_words:
        for w2 in random_words:
            f(w1, w2)
    endTime = int(round(time.time() * 1000))
    return "{t} ms".format(t=endTime - startTime)

In [7]:
print (time_on_random_words(levinstein))
print (time_on_random_words(levinstein_cached))

38842 ms
10 ms


# Decorator #2 - Json Caching

Lets try and implement a caching decorator that saves intermediate results in a json file

In [8]:
import json
import os

def json_file(fname):
    def decorator(function):
        def wrapper(*args, **kwargs):
            if os.path.isfile(fname):
                with open(fname, 'r') as f:
                    ret = json.load(f)
            else:
                with open(fname,'w') as f:
                    ret = function(*args, **kwargs)
                    json.dump(ret, f)
            return ret
        return wrapper
    return decorator

In [9]:
import time

@json_file("cached.json")
def calculation():
    time.sleep(2)
    return 1+1

In [32]:
print (calculation())

2


Using arguments for filename
---

In [33]:
from fs_cache import *

@json_file("fib_{n}.json")
def fibonacci_j(n):
    if n<2:
        return 1
    return fibonacci_j(n-1) + fibonacci_j(n-2)

fibonacci_j(5)

8

# Inspect module

Python argument types
---

In [38]:
import inspect
def sum3_named(a,b,c=3):
    return a+b+c

def sum3_positional(*args):
    return args[0]+args[1]+(args[2] if len(args)>2 else 3)

def sum3_keyvalue(**kwargs):
    return kwargs["a"]+kwargs["b"]+kwargs.get("c", 3)

Both functions have the same functionality, the only difference is the signature

In [39]:
assert sum3_named(1,2,3) == sum3_positional(1,2,3) == sum3_keyvalue(a=1,b=2,c=3)

And we can mix them up:

In [45]:
def sum3_named_and_args_and_keyval(a,*args,**kwargs):
    return a + args[0] + kwargs.get("c", 3)


When decorating a function, We would to
---
    1. Abstract away the argument type
    2. Use keyvalue argument type for our decorator

Binding function parameters
---
`signature` helps us bind positional, names, and key-value arguments, regardless of their type to another function

In [40]:
print (inspect.signature(sum3_named))
print (inspect.signature(sum3_named).bind(1,2,3).arguments)

(a, b, c=3)
OrderedDict([('a', 1), ('b', 2), ('c', 3)])


Applying `sum3_named`'s parameters to `sum3_keyvalue`

In [46]:
kwargs = inspect.signature(sum3_named).bind(1,2,3).arguments
print ( sum3_keyvalue(**kwargs))

6


Using Inspect to use filename templates
---

In [16]:
import json, os, inspect

def json_file(fname):
    def decorator(function):
        signature= inspect.signature(function)
        def wrapper(*args, **kwargs):
            file_name= fname.format(**signature.bind(*args, **kwargs).arguments)
            if os.path.isfile(file_name):
                with open(file_name, 'r') as f:
                    ret = json.load(f)
            else:
                with open(file_name,'w') as f:
                    ret = function(*args, **kwargs)
                    json.dump(ret, f)
            return ret
        return wrapper
    return decorator

# Reading a function's code:

In [17]:
print(inspect.getsource(f))

def f(a,b,c=3):
    return a+b+c



# Decorator #3 - Remote SSH

I wrapped the famous "paramiko" ssh client and made it more convinient for jupyter notebooks

In [18]:
import json
from ssh import ssh_connect

In [19]:
with open("ssh.json","r") as f:
    credentials = json.load(f)

In [20]:
myserver = ssh_connect(credentials["user"],credentials["pass"],credentials["server"])

**Get current path**

In [21]:
def local_call():
    import os
    return os.getcwd()

local_call()

'/Users/urig/Documents/AOP'

In [22]:
@myserver
def remote_call():
    import os
    return os.getcwd()

remote_call()

'/home/urig'

Equivalent to:
`remote_call=myserver(local_call)`

**Bash Execution**

In [23]:
myserver>>"pwd"

pwd
/home/urig
python-ssh:


# Decorator #4 - Regression Testing

In [24]:
from regres_test import regress
r=regress()
@r.record
def fibonacci(n):
    if n<=2:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

In [25]:
for i in range(4):
    fibonacci(i)

In [26]:
r.tests

[(OrderedDict([('n', 0)]), 1),
 (OrderedDict([('n', 1)]), 1),
 (OrderedDict([('n', 2)]), 1),
 (OrderedDict([('n', 3)]), 2)]

In [27]:
def fibonacci_formula(n):
    phi = (5**0.5+1)/2.0
    return (phi**n-(-phi)**(-n))/(5**0.5)

In [28]:
r.replay(fibonacci_formula)

Functions differ on: n=0 


# Decorator #5 - Easy Interactive Graphs

In [35]:
from ipywidgets import interact
@interact(n=(0.0,100.0))
def sqrt(n):
    return n**0.5

In [30]:
%matplotlib inline
from ipywidgets import interact
from ipykernel.pylab.backend_inline import flush_figures
from matplotlib import pyplot as plt
import numpy as np


@interact(sigma=(0.0,3.0))
def igraph(sigma):
    """Plots a gaussian whose stadtard deviation is controller from a jupyter widget"""
    x = np.linspace(-2,2,100)
    f = lambda t: np.exp(-t**2/(sigma**2))
    y = [f(p) for p in x]
    fig = plt.plot(x,y)
    plt.xlim((-2,2))
    plt.ylim((0,1))
    flush_figures()

Widget Javascript not detected.  It may not be installed or enabled properly.


All code available at:
https://github.com/urigoren/pycon2017IL